In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import random
import pandas as pd
from tqdm import tqdm
import time

## Lenta

In [ ]:
def get_request_lenta(url, proxy):
    headers = {
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'Accept-Language': 'ru-RU,ru;q=0.9',
      'Connection': 'keep-alive',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'none',
      'Sec-Fetch-User': '?1',
      'Upgrade-Insecure-Requests': '1',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
      'sec-ch-ua': '"Chromium";v="106", "Google Chrome";v="106", "Not;A=Brand";v="99"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'Cookie': 'is_mobile=0; lid=vAsAAD9GUGN8CCtiAXoSHQB=; lids=4820359A11087691'
    }
    
    response = requests.request("GET", url, headers=headers, proxies=proxy)
    return response

def get_nav_data_lenta(year, month, day, proxy):
    try:
        url = f"https://lenta.ru/{year}/{month}/{day}/"

        response = get_request_lenta(url, proxy)
        soup = BeautifulSoup(response.text, 'lxml')
        blocks = soup.find_all('li', {"class": "archive-page__item _news"})
        links = (block.find('a')['href'] for block in blocks)

        return links
    except:
        print(proxy)
        return None

def get_content_lenta(url, proxy):
    try:
        response = get_request_lenta(url, proxy)

        soup = BeautifulSoup(response.text, 'lxml')
        content = soup.find('div', {"class": "topic-body__content"})
        rubric = soup.find('div', {"class": "rubric-header__title"})
    
        return {content.get_text(), rubric.get_text()}
    except:
        #print(proxy)
        return None

In [ ]:
tmp = ["8qjf90:SGBgHV@213.166.75.141:9107",
       "8qjf90:SGBgHV@213.166.72.212:9988",
       "8qjf90:SGBgHV@213.166.74.150:9960",
       "8qjf90:SGBgHV@213.166.75.254:9096"]
pl = [{ "https" : f"https://{p}"}  for p in tmp]

In [ ]:
years = ['2012', '2013', '2015', '2016', '2017', '2018']
months = ['0'+str(m) for m in range(1,13) if m <10]
[months.append(str(m)) for m in range(1,13) if m >=10]
days = ['0'+str(d) for d in range(1,32) if d<10]
[days.append(str(d)) for d in range(1,32) if d >=10]
print(len(years), len(months), len(days))

6 12 31


In [ ]:
df_lenta = pd.DataFrame(columns = {'Resource', 'Date','Rubric','Text', 'URL'})

In [ ]:
error = [] #список для ссылок, выдающих ошибку
for year in tqdm(years):
    for month in tqdm(months):
        for day in tqdm(days):
            try:
                proxy = random.choice(pl)
                links = ["https://lenta.ru" + x for x in  get_nav_data_lenta(year,month,day,proxy)]
                for l in range(len(links)):
                    try: 
                        proxy = random.choice(pl)
                        text, rubric = get_content_lenta(links[l], proxy)
                        if len(rubric)>50:
                            text, rubric = rubric, text #тут чисто костыль потому что с чего-то вдруг для некоторых статей начали меняться местами рубрика и текст
                        df_lenta = df_lenta.append({'Resource': 'Lenta',
                                           'Date': '/'.join([year, month, day]),
                                           'Rubric': rubric,
                                           'Text': text,
                                           'URL': links[l]}, ignore_index = True)
                        if l%10==0:
                            time.sleep(1)
                    except TypeError:
                        error.append(links[l])
            except TypeError:
                pass
        df_lenta.to_parquet('df_lenta_do_{}_{}.parquet'.format(year, month))

# RBC

In [ ]:
def get_request_rbc(url, query_enc, proxy):
    headers = {
    'Host': 'www.rbc.ru',
    'Sec-Ch-Ua': '"-Not.A/Brand";v="8", "Chromium";v="102"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'Sec-Ch-Ua-Mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': f'https://www.rbc.ru/search2/?query={query_enc}',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'close',
    'Cookie': 'qrator_msid=1666207863.677.YEYJytE6RqTAhyw1-vbfsg7re8tvpssm2o8oob1pap82f0a7o; splituid=uUjlVmNQUHcF1TMvAwMjAg==; js_d=false'
  }

    response = requests.request("GET", url, headers=headers, proxies=proxy)
    return response

def get_nav_data_rbc(query, page, proxy):
    query_enc = urllib.parse.quote(query)
    url = f"https://www.rbc.ru/search2/ajax/?query={query_enc}&page={page}"
    response = get_request_rbc(url, query_enc, proxy)

    return [x.get('fronturl','') for x in response.json().get('items',[])]

def get_content_rbc(url, query, proxy):
    query_enc = urllib.parse.quote(query)

    response = get_request_rbc(url, query_enc, proxy)
    soup = BeautifulSoup(response.text, 'lxml')
    content = soup.find_all('div', {"class": "article__text article__text_free"})
    return content[0].get_text()

In [ ]:
queries = ['Россия', "Абхазия", "Белоруссия", "Беларусь", 
           "Казахстан", "Кыргызстан", "Киргизия", "Южная Осетия", 
           "Таджикистан", "Армения", "Азербайджан", "Узбекистан", "Молдова", "Молдавия"]
tmp = ["8qjf90:SGBgHV@213.166.75.141:9107",
       "8qjf90:SGBgHV@213.166.75.151:9292",
       "8qjf90:SGBgHV@213.166.72.212:9988",
       "8qjf90:SGBgHV@213.166.74.150:9960",
       "8qjf90:SGBgHV@213.166.75.254:9096",
       "8qjf90:SGBgHV@213.166.75.189:9188"]
pl = [{ "https" : f"https://{p}"}  for p in tmp]

In [ ]:
df_rbc = pd.DataFrame(columns = {'Resource','Query','Page','Rubric','Text', 'URL'})

In [ ]:
for query in tqdm(queries):
    for page in tqdm(range(1,301)): 
        try:
            proxy = random.choice(pl)
            
            #читаем ссылки
            urls_all = get_nav_data_rbc(query, page, proxy)
            urls = list(filter(lambda x: (x.startswith('https://www.rbc.ru')), urls_all))
            nontyp = list(filter(lambda x: (x not in urls),urls_all)) #забираем в отдельный список нетипичные ссылки

            #читаем текст
            for i in range(len(urls)):
                try:
                    text = get_content_rbc(urls[i], query, proxy)
                    text = text.replace("\n", "").replace('\r','')
                    #читаем рубрику
                    rubric = url.split('https://www.rbc.ru/')[1].split('/')[0]
                
        
                    df_rbc = df_rbc.append({'Query': query,
                                        'Page': page,
                                        'Rubric': rubric,
                                        'Text': text, 
                                        'URL': urls[i]}, ignore_index = True)
                    if i%40==0:
                        time.sleep(1)
                except: 
                    error.append(urls[i])
                    pass
        except requests.exceptions.ProxyError as err:
            proxy = random.choice(pl)
            continue

In [ ]:
df_rbc.to_parquet('df_rbc_colab_2.parquet')